# Entorno de trabajo: Crear un dataset de noticias de prensa a partir del motor de búsqueda Sophia

Este notebook muestra una manera de extraer noticias desde el motor de búsqueda de _Sophia_, a partir de una **palabra clave**, un **país** y un **periodo de tiempo**.


_Sophia_ es un proyecto I+i+D desarrollado en la Universidad Austral de Chile desde 2019. El motor de búsqueda se encuentra en fase de mejoramiento continuo. Por cualquier pregunta o problema técnica con este notebook, no dudar en contactarme por Discord (@MatthieuVernier) o por correo mvernier@inf.uach.cl.

## 0. Librerías útiles

- Elasticsearch es el motor de búsqueda utilizado por Sophia
- Requiere la instalación del paquete elasticsearch

In [1]:
!python -V

Python 3.9.12


In [2]:
!pip install elasticsearch

In [3]:
import elasticsearch
from datetime import datetime

In [4]:
elasticsearch.__version__

(7, 16, 3)

## 1. Parámetros de la búsqueda

Características del motor de búsqueda:  
- El formato de las fechas es: YYYY-MM-DD
- El motor de búsqueda incluye noticias por distintos paises, pero se recomienda utilizar "chile" por el momento.
- El motor de búsqueda incluye datos entre el año 2000 y el año 2021.

In [5]:
country="chile"
from_="2018-01-01"
to_="2021-12-31"
media_outlets=["elllanquihue","elaustral","laestrelladechiloe","elheraldoaustral","radiosago",
               "elrepuertero","elvacanudo","elhuemul","seminariolocal","elquellonino","elinsular",
               "radiopudeto","radioacogida","elcalbucano","segundos33",
               "prensadelestuario","fresiaahora","soychiloe"]

keyword="carabinero"
simple_keyword=False

## 2. Parámetros de conexión al motor de búsqueda de Sophia

In [6]:
IP = "search.sophia2.org"
PORT = 9200
USER = "elastic"
PASS = "dummy_elastic"

es = elasticsearch.Elasticsearch(
    f"http://{IP}:{PORT}",
    http_auth=(USER, PASS)
)

## 3. Búsqueda de noticias de prensa

- Preparación de la consulta en el formato solicitado por Elasticsearch

In [7]:
#NO MODIFICAR
query = { 
    "bool": {
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }}},
        { "term":  { "country": country }},
        { "terms":  { "media_outlet": media_outlets }} 
    ]
    }  
}

- Envio de la consulta
- No se puede devolver más de 10.000 noticias. En caso de obtener 10.000 respuestas, se recomienda buscar en un periodo de tiempo más corto o modificar la consulta.

In [8]:
res = es.search(index="news", query=query, size=1000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

Son 10000 noticias encontradas...


## 4. Guardar los datos en un dataframe Pandas y en un archivo CSV

In [11]:
import pandas as pd

data = {'id_news': [], 'title': [], 'text': [],
        'anotador': [], 'label': [], "label_ministerio": []}

df = pd.DataFrame(data)

for hit in res['hits']['hits']:
    id_news = hit['_source']['id_news']
    title = hit['_source']['title']
    text = hit['_source']['text']
    df.loc[len(df.index)] = [id_news, title, text, "spacheco", "", ""]
df


,id_news,title,text,anotador,label,label_ministerio
0,21858758,Pepsi se disculpó por publicidad sobre el cort...,La filial chilena de Pepsi ofreció disculpas p...,spacheco,,
1,21858768,Pdte Cámara de Diputados en visita a la Osorno...,"En visita a la comuna de Osorno, el presidente...",spacheco,,
2,21858769,Chiloé activó centros de acopio donde reunir a...,Luego de conocer el adverso panorama de los ha...,spacheco,,
3,21858779,Gobierno interpuso denuncia en contra de quien...,Durante el último Comité Operativo de Emergenc...,spacheco,,
4,21858781,Corte de Agua: Cámara de Comercio de Osorno pr...,"La Cámara de Comercio e Industrias, Servicios ...",spacheco,,
...,...,...,...,...,...,...
995,21869979,Región de Los Lagos se alza como un prometedor...,La región de Los Lagos siempre ha destacado co...,spacheco,,
996,21869982,Purranque se alza como polo cultural de Los La...,"Imagen: Osvaldo Villaseca, encargado de la Uni...",spacheco,,
997,21869985,Celebran el Día Mundial de la leche en Plaza d...,Imagen: SoyOsorno Junto a las actividades efec...,spacheco,,
998,21869821,Homicidio queda al descubierto en vía pública ...,Imagen: Archivo Un trágico hecho de sangre es ...,spacheco,,


- Guardar los archivos en un CSV

In [12]:
df_sample = df.sample(n=50, random_state=112233)
nombre_archivo = keyword.replace(' ', '')+"_"+country+"_"+from_+"_"+to_+".csv"
print("El archivo se llama:")
print(nombre_archivo)
df_sample.to_csv("./sebap_anotaciones_manuales.csv", sep=";")


El archivo se llama:
carabinero_chile_2018-01-01_2021-12-31.csv


- Realizar algunas consultas SQL sobre el dataset

In [48]:
from pandasql import sqldf 

In [49]:
result = sqldf("SELECT media_outlet,count(*) FROM df GROUP BY media_outlet ORDER BY count(*) DESC")
result

,media_outlet,count(*)
0,elllanquihue,465
1,elaustral,386
2,laestrelladechiloe,357
3,elvacanudo,236
4,elrepuertero,230
5,elheraldoaustral,217
6,radiosago,142
7,elhuemul,90
8,radiopudeto,42
9,seminariolocal,19
